In [2]:
# code by Tae Hwan Jung @graykode
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
dtype = torch.FloatTensor


#这个project中 X为sentence中前3个word，预测最后一个word


sentences = [ "i really like dog", "i  do love coffee", "i very hate milk","i trully memory cat"]

word_list = " ".join(sentences).split()
word_list = list(set(word_list))
word_dict = {w: i for i, w in enumerate(word_list)}
number_dict = {i: w for i, w in enumerate(word_list)}
v = len(word_dict) # number of Vocabulary


print(word_list)
print(word_dict)
print(number_dict)
print(v)



['like', 'i', 'trully', 'do', 'dog', 'love', 'coffee', 'hate', 'cat', 'milk', 'really', 'memory', 'very']
{'like': 0, 'i': 1, 'trully': 2, 'do': 3, 'dog': 4, 'love': 5, 'coffee': 6, 'hate': 7, 'cat': 8, 'milk': 9, 'really': 10, 'memory': 11, 'very': 12}
{0: 'like', 1: 'i', 2: 'trully', 3: 'do', 4: 'dog', 5: 'love', 6: 'coffee', 7: 'hate', 8: 'cat', 9: 'milk', 10: 'really', 11: 'memory', 12: 'very'}
13


In [6]:

# NNLM Parameter
max_length = 3  #max_lenth_of_input_word
hidden = 6       # hidden
d =2             #d=dimention
def make_batch(sentences):
    input_batch = []
    target_batch = []
    for sen in sentences:
        word = sen.split()
        input = [word_dict[n] for n in word[:-1]]
        target = word_dict[word[-1]]

        input_batch.append(input)
        target_batch.append(target)

    return input_batch, target_batch






# Model
class NNLM(nn.Module):
    def __init__(self):
        super(NNLM, self).__init__()
        self.C = nn.Embedding(v, d)
        self.H = nn.Parameter(torch.randn(max_length * d, hidden).type(dtype))
        self.W = nn.Parameter(torch.randn(max_length * d, v).type(dtype))
        self.d = nn.Parameter(torch.randn(hidden).type(dtype))
        self.U = nn.Parameter(torch.randn(hidden, v).type(dtype))
        self.b = nn.Parameter(torch.randn(v).type(dtype))

    def forward(self, X):
        X = self.C(X)          #(d,v) * (v,max_length,b) = (b,d,max_length)
        X = X.view(-1, max_length * d) # [b, max_length * v]
        tanh = torch.tanh(self.d + torch.mm(X, self.H)) # [batch_size, hidden]
        output = self.b + torch.mm(X, self.W) + torch.mm(tanh, self.U) # [batch_size, n_class]
        return output

model = NNLM()

print(model)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

input_batch, target_batch = make_batch(sentences)
#input_batch  (b,max_length)= (4,3)
input_batch = Variable(torch.LongTensor(input_batch))
target_batch = Variable(torch.LongTensor(target_batch))

# Training
for epoch in range(5000):
    optimizer.zero_grad()

    # print("input" ,input_batch)
    output = model(input_batch)

    # output : [batch_size, n_class], target_batch : [batch_size] (LongTensor, not one-hot)
    loss = criterion(output, target_batch)
    if (epoch + 1)%1000 == 0:
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))

    loss.backward()
    optimizer.step()



NNLM(
  (C): Embedding(13, 2)
)


Epoch: 1000 cost = 0.026569


Epoch: 2000 cost = 0.007133


Epoch: 3000 cost = 0.003072


Epoch: 4000 cost = 0.001544


Epoch: 5000 cost = 0.000833


In [8]:
# Predict
predict = model(input_batch).data.max(1, keepdim=True)[1]

# Test
print([sen.split()[:3] for sen in sentences], '->', [number_dict[n.item()] for n in predict.squeeze()])


[['i', 'really', 'like'], ['i', 'do', 'love'], ['i', 'very', 'hate'], ['i', 'trully', 'memory']] -> ['dog', 'coffee', 'milk', 'cat']
